In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_july=pd.read_csv("data/train_July.csv")
train_aug=pd.read_csv("data/train_Aug.csv")
weather=pd.read_csv("data/weather.csv")
poi=pd.read_csv("data/poi.csv")
test=pd.read_csv("data/test_id_Aug_agg_public5k.csv")

In [3]:
train_aug_dict=train_aug.groupby(['start_geo_id','end_geo_id','create_date','create_hour'])['id'].count().to_dict()
train_aug['count']=train_aug.apply(lambda x:train_aug_dict[tuple([x['start_geo_id'],x['end_geo_id'],x['create_date'],x['create_hour']])],axis=1)

In [4]:
train_aug=train_aug[train_aug[['start_geo_id','end_geo_id','create_date','create_hour']].duplicated()==False]
train_aug=train_aug[['start_geo_id','end_geo_id','create_date','create_hour','count']]

In [5]:
test.head()

,test_id,start_geo_id,end_geo_id,create_date,create_hour
0,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21
1,1,c538ad66d710f99ad0ce951152da36a4,6097a8df0b2757894ce7b4a48ae2a692,2017-08-01,21
2,2,c538ad66d710f99ad0ce951152da36a4,27d75f17e61587172fe7a6827bbaa198,2017-08-01,21
3,3,c538ad66d710f99ad0ce951152da36a4,6097a8df0b2757894ce7b4a48ae2a692,2017-08-01,17
4,4,92e1e8020813ef939183e345626b442a,92e1e8020813ef939183e345626b442a,2017-08-01,17


In [6]:
train_aug.head()

,start_geo_id,end_geo_id,create_date,create_hour,count
0,aa532f0935c3f7f5d2f1f337125998a0,940ed2bc6835c66f6261c972d4176d25,2017-08-01,12,1
1,0a64b01e91cf367a9b016d4f6d42290f,1239d1bfdc532392645d7a37f7469adf,2017-08-01,12,3
2,c538ad66d710f99ad0ce951152da36a4,c538ad66d710f99ad0ce951152da36a4,2017-08-01,14,7
3,ba20ac2de22cfe373f316eaa256b85fc,ba20ac2de22cfe373f316eaa256b85fc,2017-08-01,14,5
4,83fdd3fc77b27bdbfbc1b3a8d7912fbf,3d53327ef089e222f9d4aa6823f60c70,2017-08-01,16,1


In [7]:
merged_data=test.merge(train_aug,on=['start_geo_id','end_geo_id','create_date'],how='inner')

In [8]:
result=dict()
def get_count(x):
    if x['test_id'] in result.keys():
        return
    value=np.array([-1,-1])
    data=merged_data[merged_data['test_id']==x['test_id']]
    idx=data['create_hour_x']>data['create_hour_y']
    wt=1.0/abs(data['create_hour_x']-data['create_hour_y'])**0.7
    if sum(idx)>0:
        value[0]=sum(data['count'][idx]*wt[idx])/sum(wt[idx])
    if sum(idx==False)>0:
        value[1]=sum(data['count'][idx==False]*wt[idx==False])/sum(wt[idx==False])
    w=np.array([0.5,0.5])
    flag=value>0
    pre=sum(value[flag]*w[flag])/sum(w[flag])
    result[x['test_id']]=pre

,test_id,start_geo_id,end_geo_id,create_date,create_hour_x,create_hour_y,count
0,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,12,4
1,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,18,10
2,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,22,32
3,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,8,7
4,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,16,3
5,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,2,2
6,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,0,6
7,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,20,5
8,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,14,3
9,56,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,15,12,4


In [15]:
sub=pd.DataFrame({"test_id":test['test_id']})

In [16]:
sub['count']=test['test_id'].apply(lambda x:result[x] if x in result.keys() else 1)

In [19]:
sub.to_csv("../result/submission.csv",index=False)

In [10]:
merged_data.head()

,test_id,start_geo_id,end_geo_id,create_date,create_hour_x,create_hour_y,count
0,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,12,4
1,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,18,10
2,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,22,32
3,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,8,7
4,0,c538ad66d710f99ad0ce951152da36a4,90bb1d035e403538d20b073aec57bea2,2017-08-01,21,16,3
